## Tacotron 2 inference code 
Edit the variables **checkpoint_path** and **text** to match yours and run the entire code to generate plots of mel outputs, alignments and audio synthesis from the generated mel-spectrogram using Griffin-Lim.

In [1]:
!pip install khmer-nltk
!pip install khmer

     |████████████████████████████████| 7.0 MB 622 kB/s eta 0:00:01
     |████████████████████████████████| 78 kB 5.5 MB/s eta 0:00:011
     |████████████████████████████████| 319 kB 6.7 MB/s eta 0:00:01
  Using cached scikit_learn-1.6.1-cp39-cp39-macosx_12_0_arm64.whl (11.1 MB)
  Using cached threadpoolctl-3.6.0-py3-none-any.whl (18 kB)
  Using cached numpy-2.0.2-cp39-cp39-macosx_14_0_arm64.whl (5.3 MB)
  Using cached scipy-1.13.1-cp39-cp39-macosx_12_0_arm64.whl (30.3 MB)
  Using cached joblib-1.5.2-py3-none-any.whl (308 kB)
You should consider upgrading via the '/Users/maohieng/cadt/master/cadt_tts/phanet/tacotron2_khmer/.myenv/bin/python3 -m pip install --upgrade pip' command.
     |████████████████████████████████| 8.5 MB 841 kB/s eta 0:00:01
     |████████████████████████████████| 144 kB 1.9 MB/s eta 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
Using legacy 'setup.py install' for khmer, since 

#### Import libraries and setup matplotlib

In [1]:
import matplotlib
%matplotlib inline
import matplotlib.pylab as plt

import IPython.display as ipd

import sys
sys.path.append('waveglow/')
import numpy as np
import torch

#### Setup hparams

In [3]:
from hparams import create_hparams

hparams = create_hparams()
hparams.sampling_rate = 22050

#### Load model from checkpoint

In [22]:
from train import load_model

# checkpoint_path = "/Users/maohieng/cadt/master/cadt_tts/phanet/tacotron2_khmer/pre-training-model/checkpoint_male" #male
# checkpoint_path = "/Users/maohieng/cadt/master/cadt_tts/phanet/tacotron2_khmer/pre-training-model/checkpoint_female" #female
checkpoint_path = "/Users/maohieng/cadt/master/cadt_tts/phanet/tacotron2_khmer/pre-training-model/checkpoint_33000_khmer" #?

# Check if CUDA is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Load the checkpoint with the appropriate device
model = load_model(hparams)
checkpoint = torch.load(checkpoint_path, map_location=device)
model.load_state_dict(checkpoint['state_dict'])

if device.type == 'cuda':
    model = model.cuda().eval().half()
else:
    model = model.cpu().eval()

Using device: cpu


#### Load WaveGlow for mel2audio synthesis and denoiser

In [23]:
from denoiser import Denoiser

waveglow_path = '/Users/maohieng/cadt/master/cadt_tts/phanet/tacotron2_khmer/waveglow_256channels_universal_v5.pt'
waveglow = torch.load(waveglow_path, map_location=device, weights_only=False)['model']

if device.type == 'cuda':
    waveglow = waveglow.cuda().eval().half()
    denoiser = Denoiser(waveglow)
else:
    waveglow = waveglow.cpu().eval()
    denoiser = Denoiser(waveglow)

if device.type == 'cuda':
    for k in waveglow.convinv:
        k.float()

#### Prepare text input

In [24]:
# text = "រោងចក្រ នេះ បាន ផលិត ស្រាបៀរ ដោយ ដាក់ ស្លាក សញ្ញា ពីរ គឺ ស្រាបៀរ អង្គរ និង ស្រាបៀរ បាយ័ន ។"
# text = "មន្ត្រី អង្គការ ក្រៅ រដ្ឋាភិបាល បាន ដាក់ ពាក្យ បណ្ដឹង ទៅ តុលាការ ខេត្ត ក្រចេះ ។"
# text = "ខ្ញុំ ត្រូវបាន ឡោមព័ទ្ធ ដោយ ក្មេង ក្មេង ហើយ ពួកវា បាន ធ្វើឱ្យ ការ៉េម ប្រឡាក់ ខោអាវ របស់ ខ្ញុំ ។"
# text = "ស៊ុត មួយ ចាន ក្រែម បង្គា ចៀន ជាមួយ ក្រូចឆ្មារ ប៊ឺ ដំឡូង ជីវ៉ាន់ ស៊ុយ សាឡាដ បៃតង សាច់ មូល និង កាហ្វេ ឬ តែ ។"
# text = "សុំទោស លោក យ៉ាស៊ី ប៉ុន្តែ លោក ឃែន កំពុងតែ និយាយ ទូរសព្ទ ជាមួយ ខេប ថោន ។"
# text = "តើ ហេតុ អ្វី បានជា អ្នក មិន ប្រើ វា នៅពេល ធ្វើតេស្ត បើកបរ ?"
# text = "សូម សួរ រក សំបុត្រ យប់ នេះ នៅ ការិយាល័យ លក់ សំបុត្រ ពេល ថ្ងៃ ។"
# text = "នៅ ពេលណា ដែល ខ្ញុំ ចរចា តម្លៃ ខ្ញុំ មិន អាច រក អ្វី ដែល ខ្ញុំ ចូលចិត្ត ហើយ ត្រូវ នឹង ទំហំ របស់ ខ្ញុំ នោះ ទេ ។"
# text = "បន្ទាប់ពី ហូប អាហារ នៅ ភោជនីយដ្ឋាន ហើយ ខ្ញុំ បាន ដើរ នៅ កន្លែង លំហែ ។"
# text = "ខ្ញុំ សង្ឃឹម ថា អ្នក នឹង រីករាយ ជាមួយនឹង ការបង្រៀន របស់ គាត់ ហើយ ទទួលបាន នូវ អ្វី ដែល អាច ជួយ ឱ្យ ជីវិត របស់ អ្នក រីកចំរើន ។"
# text = "ខ្ញុំ បាន សន្យា ជាមួយ ខ្លួនឯង ថា នៅ ពេល ដែល ខ្ញុំ ជួប អ្នក ខ្ញុំ នឹង ធ្វើឱ្យ អ្នក ស្គាល់ ខ្ញុំ ។" 
# text = "សុំទោស នេះ មិន មែន ជា របស់ ដែល ខ្ញុំ បាន បញ្ជាទិញ នោះ ទេ ។" 
# text = "យើង ទទួល តែ សែក ធ្វើដំណើរ និង កាត ឥណទាន ប៉ុណ្ណោះ ។" 
# text = "សូម ទូរសព្ទ មក ខ្ញុំ ដោយ កុំ ក្រែង ចិត្ត ថ្វី បើ មាន ឱកាស មក លេង តូក្យូ ។" 
text = "វា នឹង មាន រសជាតិ កាន់តែ ល្អ ប្រសើរ ពេល ដែល ជ្រលក់ ជាមួយ ទឹក ស៊ីអ៊ីវ ។" 

In [25]:
import run

text = run.text_process(text)
input = run.textNorm(text)

វា   នឹង   មាន   រសជាតិ   កាន់តែ   ល្អ   ប្រសើរ   ពេល   ដែល   ជ្រលក់   ជាមួយ   ទឹក   ស៊ីអ៊ីវ   ។;


In [26]:
from text import text_to_sequence

sequence = np.array(text_to_sequence(input, ['english_cleaners']))[None, :]
sequence = torch.autograd.Variable(
    torch.from_numpy(sequence)).to(device).long()

#### Decode text input and plot results

In [27]:
def plot_data(data, figsize=(16, 4)):
    fig, axes = plt.subplots(1, len(data), figsize=figsize)
    for i in range(len(data)):
        axes[i].imshow(data[i], aspect='auto', origin='lower', 
                       interpolation='none')

In [28]:
mel_outputs, mel_outputs_postnet, _, alignments = model.inference(sequence)
plot_data((mel_outputs.float().data.cpu().numpy()[0],
           mel_outputs_postnet.float().data.cpu().numpy()[0],
           alignments.float().data.cpu().numpy()[0].T))

#### Synthesize audio from spectrogram using WaveGlow

In [29]:
with torch.no_grad():
    audio = waveglow.infer(mel_outputs_postnet, sigma=0.666)
ipd.Audio(audio[0].data.cpu().numpy(), rate=hparams.sampling_rate)

#### (Optional) Remove WaveGlow bias